# Submission 1: Pengembangan Machine Learning Pipeline  

**Nama:** Mohammad Iqbal Jaffar  
**Username Dicoding:** MOHAMMAD IQBAL JAFFAR  

### Sumber Dataset  
[Kaggle - Fake News Football](https://www.kaggle.com/datasets/shawkyelgendy/fake-news-football)  


# 1. Import Library and Dependency

Kode ini bertujuan untuk mengatur lingkungan pengembangan, menginstal dependensi yang diperlukan, serta mengimpor berbagai pustaka yang digunakan dalam pengembangan Machine Learning Pipeline menggunakan TensorFlow Extended (TFX).

In [1]:
# conda deactivate
# conda remove --name mlops-tfx --all
# conda create -n mlops-tfx python=3.9.15 -y
# conda activate mlops-tfx
# pip install --upgrade pip 
# pip install -r requirements.txt
# conda install ipykernel

- **Mengelola lingkungan Conda**: Menonaktifkan (`conda deactivate`), menghapus (`conda remove`), membuat ulang (`conda create`), dan mengaktifkan kembali (`conda activate`) lingkungan `mlops-tfx` dengan Python 3.9.15.  
- **Menginstal dependensi**: Memperbarui `pip`, menginstal pustaka dari `requirements.txt`, serta menambahkan `ipykernel` untuk mendukung Jupyter Notebook.

In [2]:
# Importing necessary libraries
import os
import pandas as pd

# TensorFlow and TensorFlow Model Analysis
import tensorflow as tf
import tensorflow_model_analysis as tfma  # For model evaluation
from keras_tuner import RandomSearch # For hyperparameter tuning

# Importing TFX components
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator
from tfx.components import Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2

# TFX Interactive Context
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Resolver for resolving inputs in pipelines
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy

# TFX standard artifact types
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

# Requests and PrettyPrinter for external data handling and debugging
import requests
from pprint import PrettyPrinter

 - **Mengimpor pustaka dasar**: `os` untuk operasi sistem dan `pandas` untuk manipulasi data.  
- **Mengimpor TensorFlow dan evaluasi model**: `tensorflow` untuk membangun model, `tensorflow_model_analysis (tfma)` untuk mengevaluasi model, dan `keras_tuner` untuk hyperparameter tuning.  
- **Mengimpor komponen TFX**: Digunakan untuk membangun pipeline, termasuk `CsvExampleGen` (mengambil data dari CSV), `StatisticsGen` (menganalisis statistik data), `SchemaGen` (membuat skema data), `ExampleValidator` (memvalidasi data), `Transform` (mentransformasikan fitur), `Trainer` (melatih model), `Tuner` (melakukan hyperparameter tuning), `Evaluator` (mengevaluasi model), dan `Pusher` (mendeploy model).  
- **Mengatur TFX Interactive Context**: `InteractiveContext` digunakan untuk menjalankan pipeline dalam mode interaktif.  
- **Resolver dan strategi input pipeline**: `Resolver` digunakan untuk menangani input pipeline, dan `LatestBlessedModelStrategy` memastikan model terbaru yang telah divalidasi digunakan dalam pipeline.  
- **Mengelola artifact model**: `Channel` digunakan untuk mendefinisikan saluran artifact, dengan `Model` dan `ModelBlessing` sebagai tipe artifact standar.  
- **Mengimpor `requests` dan `PrettyPrinter`**: `requests` digunakan untuk mengambil data dari sumber eksternal, sementara `PrettyPrinter` membantu dalam debugging dengan mencetak output yang lebih terbaca.  

# 2. Set Variable

Kode ini digunakan untuk menetapkan variabel penting dalam pipeline machine learning untuk proyek *Fake News Football*, termasuk penamaan pipeline dan model, direktori penyimpanan, serta jalur data dan modul yang digunakan.  

In [3]:
# Mendefinisikan nama pipeline dan model yang digunakan
PIPELINE_NAME = "Fake_News_football_pipeline"  # Nama utama pipeline
SCHEMA_PIPELINE_NAME = "Fake_News_football_tfdv_schema"  # Nama pipeline untuk skema data
MODEL_NAME = "Fake_News_football_model"  # Nama model yang akan digunakan

# Menentukan direktori untuk menyimpan artefak pipeline, metadata, dan model yang telah dilatih
PIPELINE_ROOT = os.path.join("MOHAMMAD-IQBAL-JAFFAR-Fake_News_Footballs-pipeline", PIPELINE_NAME)  # Direktori utama pipeline
METADATA_PATH = os.path.join("metadata", PIPELINE_NAME, "metadata.db")  # Lokasi metadata pipeline
SERVING_MODEL_DIR = os.path.join("serving_model_dir", MODEL_NAME)  # Direktori tempat model tersimpan setelah deployment

# Menentukan lokasi dataset dan modul yang digunakan dalam pipeline
DATA_ROOT = "data"  # Direktori utama dataset
TRANSFORM_MODULE_FILE = "modules/Fake_News_football_transform.py"  # Modul untuk preprocessing data
TRAINER_MODULE_FILE = "modules/Fake_News_football_trainer.py"  # Modul untuk pelatihan model
TUNER_MODULE_FILE = "modules/Fake_News_football_tuner.py"  # Modul untuk tuning hyperparameter

# Menentukan direktori utama proyek dan memastikan direktori module tersedia
PROJECT_ROOT = "C:/Users/M Iqbal/a443-MLOps-ML-Pipeline"  # Direktori utama proyek
MODULES_DIR = os.path.join(PROJECT_ROOT, "modules")  # Direktori untuk menyimpan modul pipeline
os.makedirs(MODULES_DIR, exist_ok=True)  # Membuat direktori modules jika belum ada

# Menginisialisasi InteractiveContext untuk menjalankan pipeline dalam mode interaktif
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

- **Menetapkan nama pipeline dan model**: `PIPELINE_NAME`, `SCHEMA_PIPELINE_NAME`, dan `MODEL_NAME` digunakan untuk mendefinisikan nama pipeline, skema data, dan model yang akan dibuat.  
- **Menentukan direktori utama**:  
  - `PIPELINE_ROOT`: Direktori utama tempat pipeline akan disimpan.  
  - `METADATA_PATH`: Jalur penyimpanan metadata pipeline.  
  - `SERVING_MODEL_DIR`: Direktori tempat model yang telah dilatih akan disimpan untuk deployment.  
- **Menentukan jalur input data dan modul**:  
  - `DATA_ROOT`: Direktori utama tempat dataset disimpan.  
  - `TRANSFORM_MODULE_FILE`: Jalur file Python untuk tahap *Transform*.  
  - `TRAINER_MODULE_FILE`: Jalur file Python untuk tahap *Trainer*.  
  - `TUNER_MODULE_FILE`: Jalur file Python untuk *Hyperparameter Tuning*.  
- **Menyiapkan direktori proyek**:  
  - `PROJECT_ROOT`: Direktori utama proyek.  
  - `MODULES_DIR`: Direktori untuk menyimpan modul Python yang diperlukan dalam pipeline, dibuat menggunakan `os.makedirs()` agar dipastikan selalu ada.  
- **Membuat *Interactive Context***: `interactive_context` digunakan untuk menjalankan pipeline dalam mode interaktif dengan menentukan `pipeline_root` sebagai jalur utama pipeline.  

# 3. Processing Dataset with Pandas

Kode ini bertujuan untuk memproses dataset *Fake News Football* menggunakan **Pandas**, termasuk membaca data, sampling, penambahan label, penggabungan dataset, pembersihan data, penyimpanan ke file CSV, dan verifikasi hasil.  

- **Membaca dataset**: `real.csv` dan `fake.csv` dimuat ke dalam *DataFrame* `real_data` dan `fake_data` menggunakan `pd.read_csv()`.  

In [4]:
# Membaca dataset berita asli (real news) dari file CSV
real_data = pd.read_csv("real.csv")
real_data.head()  # Menampilkan 5 baris pertama dari dataset real news

,tweet
0,sun downs technical director: al-ahly respecte...
1,shawky gharib after the tie with enppi: our go...
2,"egyptian sports news today, wednesday 1/25/202..."
3,the main referees committee of the egyptian fo...
4,"haji bari, the striker of the future team, is ..."


In [5]:
# Membaca dataset berita palsu (fake news) dari file CSV
fake_data = pd.read_csv("fake.csv")
fake_data.head()  # Menampilkan 5 baris pertama dari dataset fake news

,tweet
0,"the tongue of his condition, now i saw things ..."
1,"by god the great, after i saw the derby of wyd..."
2,"believe in god, this zamalek fans are as good ..."
3,the coach appeared in the media and told the t...
4,asno smito\nwhoever knows her tells me a massa...


In [6]:
# Ambil 5000 sampel dari masing-masing kategori
real_data = real_data.sample(n=1000, random_state=42)
fake_data = fake_data.sample(n=1000, random_state=42)

# Tambahkan label dalam bentuk string terlebih dahulu
real_data["class"] = "real"
fake_data["class"] = "fake"

# Mapping label ke angka
label_mapping = {"real": 1, "fake": 0}
real_data["class"] = real_data["class"].map(label_mapping)
fake_data["class"] = fake_data["class"].map(label_mapping)

# Gabungkan dataset
df = pd.concat([real_data, fake_data], axis=0).reset_index(drop=True)

# Hapus baris yang memiliki nilai kosong di kolom "tweet"
df = df.dropna(subset=["tweet"])

# Pastikan hanya menyimpan kolom yang diperlukan
if all(col in df.columns for col in ["title", "date", "subject"]):
    df = df.drop(["title", "date", "subject"], axis=1)

# Ganti nama kolom label
df.rename(columns={"class": "real_news"}, inplace=True)

# Simpan dataset ke dalam folder "data/csv"
output_dir = "data"
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir, "news.csv"), index=False)

print("File berhasil disimpan di:", os.path.join(output_dir, "news.csv"))
print("Jumlah data setelah menghapus NaN:", len(df))

File berhasil disimpan di: data\news.csv
Jumlah data setelah menghapus NaN: 2000


- **Mengambil sampel data**: Masing-masing dataset diambil 1000 sampel secara acak (`random_state=42`) untuk menjaga replikasi hasil.  
- **Menambahkan label kategori**:  
  - Data berita asli (`real_data`) diberi label `"real"` dan berita palsu (`fake_data`) diberi label `"fake"`.  
  - Label tersebut kemudian dikonversi menjadi angka (`1` untuk berita asli, `0` untuk berita palsu) menggunakan `map()`.  
- **Menggabungkan dataset**: Kedua *DataFrame* digabungkan menjadi satu (`df`) menggunakan `pd.concat()` dan indeksnya di-*reset* dengan `reset_index(drop=True)`.  
- **Membersihkan data**:  
  - Baris dengan nilai kosong (`NaN`) pada kolom "tweet" dihapus menggunakan `dropna(subset=["tweet"])`.  
  - Jika dataset mengandung kolom `"title"`, `"date"`, dan `"subject"`, maka kolom tersebut akan dihapus.  
- **Mengganti nama kolom label**: Kolom `"class"` diubah namanya menjadi `"real_news"` agar lebih jelas.  
- **Menyimpan dataset yang telah diproses**: Dataset akhir disimpan dalam folder `"data/csv"` dengan nama `"news.csv"` menggunakan `df.to_csv()`, memastikan direktori `"data"` tersedia dengan `os.makedirs()`.  

In [7]:
# Membaca dataset yang telah diproses sebelumnya (gabungan real dan fake news)
data_news = pd.read_csv("data/news.csv")
data_news.head()  # Menampilkan 5 baris pertama dari dataset gabungan

,tweet,real_news
0,zamalek continues its preparations for the el ...,1
1,"sheikh salem al-amry, vice president of the ar...",1
2,the arab contractors tested 3 foreigners under...,1
3,an urgent decision for caf today regarding the...,1
4,al-masry refuses to postpone the al-ahly match...,1


- **Menampilkan informasi dataset**:  

In [8]:
# Menampilkan informasi struktur dataset (jumlah baris, kolom, tipe data, dan nilai yang hilang)
data_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      2000 non-null   object
 1   real_news  2000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


  - Menampilkan pesan konfirmasi penyimpanan file dan jumlah data setelah pembersihan.  
  - Memuat kembali dataset (`data_news`) dan menampilkan beberapa baris pertama dengan `head()`.  
  - Mengecek struktur dataset menggunakan `info()`, yang menunjukkan terdapat **2000 data**, dengan **1 kolom teks ("tweet")** dan **1 kolom label numerik ("real_news")**.

# 4. CsvExampleGen: Membaca dan membagi dataset

Kode ini menggunakan **CsvExampleGen**, komponen dari **TFX**, untuk membaca dataset yang telah diproses dan membaginya ke dalam set pelatihan dan evaluasi.  

In [9]:
# Konfigurasi ExampleGen
output = example_gen_pb2.Output( # Definisikan konfigurasi keluaran untuk pembagian data
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=9), # 90% data untuk pelatihan
        example_gen_pb2.SplitConfig.Split(name='eval',  hash_buckets=1) # 10% data untuk evaluasi
    ])
)

# Komponen CsvExampleGen untuk memproses data input dan menerapkan konfigurasi keluaran
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output) # Gunakan 

- **Mengonfigurasi pembagian dataset**:  
  - `example_gen_pb2.Output` digunakan untuk menentukan **proporsi data** yang akan dibagi.  
  - `split_config` mendefinisikan dua bagian:  
    - **90% data** untuk pelatihan (`train`, `hash_buckets=9`).  
    - **10% data** untuk evaluasi (`eval`, `hash_buckets=1`).  
- **Membuat komponen CsvExampleGen**:  
  - `CsvExampleGen(input_base=DATA_ROOT, output_config=output)` digunakan untuk membaca dataset dari **DATA_ROOT** dan menerapkan konfigurasi pembagian data yang telah ditentukan. 
  
  ___

- **Menjalankan pipeline secara interaktif**:  
  - `interactive_context.run(example_gen)` mengeksekusi komponen `example_gen` dalam **mode interaktif**, memungkinkan pipeline memproses data input dan membagi dataset secara otomatis. 

In [10]:
# Menjalankan komponen example_gen untuk memulai pipeline
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 7
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# 5. Membuat Tahapan Data Validation

## 5.1 StatisticsGen: Menghasilkan statistik deskriptif

 - **Membuat dan menjalankan StatisticsGen**:  
  - `StatisticsGen(examples=example_gen.outputs["examples"])` digunakan untuk menghitung **statistik deskriptif** dari dataset.  
  - `interactive_context.run(statistics_gen)` menjalankan komponen ini dalam pipeline.  
  - `interactive_context.show(statistics_gen.outputs["statistics"])` menampilkan hasil statistik yang telah dihitung.  

In [11]:
# Membuat komponen StatisticsGen untuk menghasilkan statistik deskriptif dari dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
# Menjalankan komponen StatisticsGen untuk menghitung statistik data
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 8
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [12]:
# Menampilkan statistik data yang dihasilkan oleh StatisticsGen
interactive_context.show(statistics_gen.outputs["statistics"])

## 5.2 SchemaGen: Membuat schema data secara otomatis

- **Membuat dan menjalankan SchemaGen**:  
  - `SchemaGen(statistics=statistics_gen.outputs["statistics"])` digunakan untuk menghasilkan **skema data** berdasarkan statistik yang diperoleh.  
  - `interactive_context.run(schema_gen)` mengeksekusi komponen SchemaGen dalam pipeline.  
  - `interactive_context.show(schema_gen.outputs["schema"])` menampilkan hasil skema yang dibuat, yang mencakup informasi **tipe fitur**, **kehadiran data (Presence)**, serta **domain nilai**.  

In [13]:
# Membuat komponen SchemaGen untuk menghasilkan skema data berdasarkan statistik yang telah dihitung
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
# Menjalankan komponen SchemaGen untuk menghasilkan skema dari dataset
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 9
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [14]:
# Menampilkan skema data yang dihasilkan oleh SchemaGen
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'real_news',INT,required,,-
'tweet',BYTES,required,,-



- **Ringkasan skema data yang dihasilkan**:  
  - **Fitur `real_news`** bertipe **integer** (`INT`), bersifat **required** (wajib ada).  
  - **Fitur `tweet`** bertipe **bytes** (`BYTES`), juga bersifat **required**.  

  ___

## 5.3 ExampleValidator: Mengidentifikasi anomali pada dataset

- **Membuat dan menjalankan ExampleValidator**:  
  - `ExampleValidator(statistics=statistics_gen.outputs["statistics"], schema=schema_gen.outputs["schema"])` digunakan untuk **memeriksa anomali atau ketidaksesuaian dalam dataset** dengan membandingkan data terhadap skema yang telah dibuat.  
  - `interactive_context.run(example_validator)` mengeksekusi komponen ini untuk mendeteksi anomali dalam dataset.  
  - `interactive_context.show(example_validator.outputs["anomalies"])` menampilkan hasil deteksi anomali dari dataset.  

In [15]:
# Membuat komponen ExampleValidator untuk memeriksa anomali atau ketidaksesuaian dalam dataset
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],  # Menggunakan statistik dari StatisticsGen
    schema=schema_gen.outputs["schema"],  # Menggunakan skema yang dihasilkan oleh SchemaGen
)
# Menjalankan komponen ExampleValidator untuk mendeteksi anomali dalam dataset
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 10
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [16]:
# Menampilkan hasil deteksi anomali dari ExampleValidator
interactive_context.show(example_validator.outputs["anomalies"])

# 6. Transform: Data Processing

Kode ini menggunakan komponen **Transform** dalam **TFX** untuk menerapkan preprocessing pada dataset sebelum digunakan dalam pelatihan model.  

- **Membuat file transformasi**:  
  - `%%writefile {TRANSFORM_MODULE_FILE}` menyimpan kode preprocessing ke dalam file modul transformasi.  
  - Fungsi `transformed_name(key)` menambahkan sufiks `_xf` pada fitur yang telah ditransformasi.  
  - Fungsi `preprocessing_fn(inputs)` melakukan **normalisasi teks** (mengubah huruf menjadi kecil) dan **konversi label** ke tipe integer (`int64`).  

In [17]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
LABEL_KEY = "real_news"  # Kunci untuk label
FEATURE_KEY = "tweet"  # Kunci untuk fitur teks

def transformed_name(key):
    """Menambahkan '_xf' pada nama fitur atau label untuk menunjukkan transformasi."""
    return f"{key}_xf"

def preprocessing_fn(inputs):
    """
    Melakukan preprocessing pada fitur input menjadi fitur yang telah ditransformasi.
    
    Args:
        inputs: peta dari kunci fitur ke fitur mentah.
    
    Return:
        outputs: peta dari kunci fitur ke fitur yang telah ditransformasi.
    """
    outputs = {}
    
    # Mengonversi kalimat menjadi huruf kecil untuk normalisasi
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    # Mengonversi label menjadi tipe integer
    outputs[transformed_name(LABEL_KEY)]   = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Overwriting modules/Fake_News_football_transform.py


- **Menjalankan Transform**:  
  - `Transform(examples=example_gen.outputs["examples"], schema=schema_gen.outputs["schema"], module_file=os.path.abspath(TRANSFORM_MODULE_FILE))` menghubungkan dataset dengan skema dan menerapkan preprocessing yang telah didefinisikan.  
  - `interactive_context.run(transform)` mengeksekusi Transform untuk **memproses data** sebelum pelatihan model.  

In [18]:
# Komponen Transform untuk menerapkan transformasi data
transform = Transform(
    examples=example_gen.outputs["examples"],  # Menggunakan data dari ExampleGen
    schema=schema_gen.outputs["schema"],  # Menggunakan schema dari SchemaGen
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)  # File modul transformasi
)

# Jalankan Transform menggunakan konteks interaktif
interactive_context.run(transform)  # Melakukan transformasi pada data

INFO:tensorflow:Assets written to: MOHAMMAD-IQBAL-JAFFAR-Fake_News_Footballs-pipeline\Fake_News_football_pipeline\Transform\transform_graph\11\.temp_path\tftransform_tmp\56d361a159a34920a3d5d028bb151b38\assets


INFO:tensorflow:Assets written to: MOHAMMAD-IQBAL-JAFFAR-Fake_News_Footballs-pipeline\Fake_News_football_pipeline\Transform\transform_graph\11\.temp_path\tftransform_tmp\56d361a159a34920a3d5d028bb151b38\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 11
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# 7. Tuner: Hyperparameter Tuning

Kode ini menggunakan komponen **Tuner** dalam **TFX** untuk melakukan **hyperparameter tuning** pada model LSTM menggunakan **KerasTuner**.  

- **Membuat file tuner**:  
  - `%%writefile {TUNER_MODULE_FILE}` menyimpan kode tuning ke dalam file modul.  
  - Fungsi `model_builder(hp)` mendefinisikan arsitektur model LSTM dengan **embedding layer, bidirectional LSTM, dense layers, dropout**, dan **sigmoid activation**, dengan hyperparameter yang dapat disesuaikan.  
  - Fungsi `input_fn()` memuat dataset dalam format **TFRecord** menggunakan fitur yang telah ditransformasikan.  
  - Fungsi `tuner_fn(fn_args)` melakukan tuning hyperparameter menggunakan **RandomSearch**, dengan **val_binary_accuracy** sebagai metrik utama dan early stopping sebagai callback.  

In [19]:
%%writefile {TUNER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from keras_tuner.engine import base_tuner
from typing import NamedTuple, Dict, Text, Any
from keras_tuner import RandomSearch
from tfx.components import Tuner, Trainer
from tfx.proto import trainer_pb2

LABEL_KEY = 'real_news'
FEATURE_KEY = 'tweet'

# Utility function to rename transformed features
def transformed_name(key):
    return key + "_xf"

# Utility function to read gzip files
def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

# Input function for creating datasets
def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY)
    )
    return dataset

VOCAB_SIZE = 5000
SEQUENCE_LENGTH = 1500

vectorize_layer = layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)

# Model builder for hyperparameter tuning
def model_builder(hp):
    """Build machine learning model"""
    embedding_dim = hp.Choice('embedding_dim', values=[32, 128])
    lstm_units = hp.Choice('lstm_units', values=[32, 128])
    dense_units = hp.Int('dense_units', min_value=32, max_value=128, step=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.3, step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    num_layers = hp.Int('num_layers', min_value=4, max_value=8, step=1)  # Added num_layers here

    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)

    for _ in range(num_layers):  # Use num_layers here
        x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    return model

TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner),
    ('fit_kwargs', Dict[Text, Any]),
])

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor  = 'val_binary_accuracy',
    mode     = 'max',
    verbose  = 1,
    patience = 2
)

# Tuner function
def tuner_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, num_epochs=3, batch_size=64)
    val_set = input_fn(fn_args.eval_files[0], tf_transform_output, num_epochs=3, batch_size=64)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)] for i in list(train_set)
        ]]
    )

    model_tuner = RandomSearch(
        hypermodel=model_builder,
        objective=kt.Objective('val_binary_accuracy', direction='max'),
        max_trials=4,
        executions_per_trial=2,
        directory=fn_args.working_dir,
        project_name='suicide_text_random_tuner',
    )

    return TunerFnResult(
        tuner=model_tuner,
        fit_kwargs={
            'x': train_set,
            'validation_data': val_set,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps,
            'callbacks': [
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_binary_accuracy',
                    mode='max',
                    patience=2,
                    verbose=1
                )
            ]
        }
    )

Overwriting modules/Fake_News_football_tuner.py


- **Menjalankan Tuner**:  
  - `Tuner(module_file=os.path.abspath(TUNER_MODULE_FILE), ...)` menghubungkan dataset dengan pipeline TFX untuk mencari kombinasi hyperparameter terbaik.  
  - `interactive_context.run(tuner)` mengeksekusi tuning model untuk mendapatkan konfigurasi optimal sebelum melatih model final.  

In [20]:
# Pipeline Tuner Component
tuner = Tuner(
    module_file     = os.path.abspath(TUNER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(tuner)

Trial 4 Complete [00h 13m 33s]
val_binary_accuracy: 0.7436548173427582

Best val_binary_accuracy So Far: 0.913705587387085
Total elapsed time: 01h 27m 45s
Results summary
Results in MOHAMMAD-IQBAL-JAFFAR-Fake_News_Footballs-pipeline\Fake_News_football_pipeline\.temp\12\suicide_text_random_tuner
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
embedding_dim: 128
lstm_units: 32
dense_units: 128
dropout_rate: 0.2
learning_rate: 0.001
num_layers: 7
Score: 0.913705587387085

Trial 2 summary
Hyperparameters:
embedding_dim: 128
lstm_units: 32
dense_units: 80
dropout_rate: 0.1
learning_rate: 0.001
num_layers: 7
Score: 0.9010152220726013

Trial 3 summary
Hyperparameters:
embedding_dim: 32
lstm_units: 32
dense_units: 112
dropout_rate: 0.1
learning_rate: 0.0001
num_layers: 6
Score: 0.7436548173427582

Trial 1 summary
Hyperparameters:
embedding_dim: 32
lstm_units: 32
dense_units: 96
dropout_rate: 0.1
learning_rate: 1e-05
num_layers: 7


ExecutionResult(
    component_id: Tuner
    execution_id: 12
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# 8. Trainer: Model Development

Kode ini merupakan modul **Trainer** dalam **TFX** yang digunakan untuk melatih model **LSTM** berbasis **TensorFlow** menggunakan dataset teks yang telah ditransformasikan, dengan fitur utama sebagai berikut:  

- **Membuat file trainer**:  
  - `%%writefile {TRAINER_MODULE_FILE}` menyimpan kode trainer ke dalam file modul.  
  - Fungsi `input_fn()` memuat dataset dalam format **TFRecord**, membaca fitur yang telah ditransformasikan, dan membentuk **batch dataset** untuk pelatihan model.  
  - Fungsi `model_builder(hp)` membangun model **LSTM** dengan **embedding layer, bidirectional LSTM, dense layers, dropout**, dan **sigmoid activation**, menggunakan hyperparameter yang dihasilkan dari tuning.  
  - Fungsi `_get_serve_tf_examples_fn()` mendefinisikan fungsi untuk melakukan inferensi pada model yang telah dilatih.  

In [21]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "real_news"
FEATURE_KEY = "tweet"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    
    # # Add dataset repeat operation
    # dataset = dataset.repeat()  # Ensure dataset repeats indefinitely or for specified epochs
    return dataset
 

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 120
embedding_dim = 16

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 

def model_builder(hp):
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp['embedding_dim'], name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(hp['lstm_units']))(x)
    for _ in range(hp['num_layers']):
        x = layers.Dense(hp['dense_units'], activation='relu')(x)
    x = layers.Dropout(hp['dropout_rate'])(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.BinaryCrossentropy(from_logits=False),
        optimizer = tf.keras.optimizers.Adam(hp['learning_rate']),
        metrics   = [tf.keras.metrics.BinaryAccuracy()]
    )
    
    model.summary()
    return model
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
# Trainer function
def run_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=2, batch_size=64)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=2, batch_size=64)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)] for i in list(train_set)
        ]]
    )

    hp = fn_args.hyperparameters['values']
    model = model_builder(hp)

    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')),
        tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=10, mode='max', verbose=1),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=fn_args.serving_model_dir,
            monitor='val_binary_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        )
    ]

    model.fit(
        x=train_set,
        validation_data=val_set,
        epochs=2,
        steps_per_epoch=fn_args.train_steps,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks
    )

    signatures = {
        'serving_default': tf.function(
            model
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
        )
    }

    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Writing modules/Fake_News_football_trainer.py


- **Menjalankan Trainer**:  
  - Fungsi `run_fn(fn_args)` mengambil dataset pelatihan dan validasi, melakukan adaptasi **TextVectorization**, lalu melatih model menggunakan **Adam optimizer** dengan metrik **binary accuracy**.  
  - Proses pelatihan menggunakan **callbacks** seperti **TensorBoard logging, early stopping**, dan **model checkpointing** untuk menyimpan model terbaik.  
  - Model yang telah dilatih disimpan dalam format **TensorFlow SavedModel** dengan signature untuk inferensi.  
  - `Trainer(...)` menghubungkan dataset dengan pipeline TFX, menggunakan hyperparameter terbaik dari tuning sebelumnya.  
  - `interactive_context.run(trainer)` mengeksekusi proses pelatihan model dalam pipeline TFX.  

In [22]:
trainer = Trainer(
    module_file     = os.path.abspath(TRAINER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    hyperparameters = tuner.outputs['best_hyperparameters'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tweet_xf (InputLayer)       [(None, 1)]               0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 120)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 120, 128)          1280000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense_7 (Dense)             (None, 128)               8320

INFO:tensorflow:Assets written to: MOHAMMAD-IQBAL-JAFFAR-Fake_News_Footballs-pipeline\Fake_News_football_pipeline\Trainer\model\13\Format-Serving\assets


57/57 [==============================] - 92s 1s/step - loss: 0.5216 - binary_accuracy: 0.7049 - val_loss: 0.2596 - val_binary_accuracy: 0.9036
Epoch 2/2
57/57 [==============================] - ETA: 0s - loss: 0.1624 - binary_accuracy: 0.9387
Epoch 2: val_binary_accuracy improved from 0.90355 to 0.92893, saving model to MOHAMMAD-IQBAL-JAFFAR-Fake_News_Footballs-pipeline\Fake_News_football_pipeline\Trainer\model\13\Format-Serving
INFO:tensorflow:Assets written to: MOHAMMAD-IQBAL-JAFFAR-Fake_News_Footballs-pipeline\Fake_News_football_pipeline\Trainer\model\13\Format-Serving\assets


INFO:tensorflow:Assets written to: MOHAMMAD-IQBAL-JAFFAR-Fake_News_Footballs-pipeline\Fake_News_football_pipeline\Trainer\model\13\Format-Serving\assets


57/57 [==============================] - 91s 2s/step - loss: 0.1624 - binary_accuracy: 0.9387 - val_loss: 0.2107 - val_binary_accuracy: 0.9289
INFO:tensorflow:Assets written to: MOHAMMAD-IQBAL-JAFFAR-Fake_News_Footballs-pipeline\Fake_News_football_pipeline\Trainer\model\13\Format-Serving\assets


INFO:tensorflow:Assets written to: MOHAMMAD-IQBAL-JAFFAR-Fake_News_Footballs-pipeline\Fake_News_football_pipeline\Trainer\model\13\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 13
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# 9. Model Analysis and Validation

## 9.1 Resolver

- **Model Resolver**:  
  - `Resolver` digunakan untuk mengambil model terbaik sebelumnya yang telah diberkati (**blessed model**) menggunakan **LatestBlessedModelStrategy**, yang memastikan hanya model terbaik yang dipilih sebagai baseline untuk evaluasi.  

In [23]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 14
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## 9.2 Evaluator


- **Evaluasi Model**:  
  - `EvalConfig` mendefinisikan konfigurasi evaluasi menggunakan **TensorFlow Model Analysis (TFMA)**, termasuk **label key, slicing specs**, dan berbagai metrik evaluasi seperti **AUC, binary accuracy, false positives, dan true positives**.  
  - `Evaluator` membandingkan model baru dengan **baseline model** untuk menilai kinerjanya berdasarkan konfigurasi yang telah ditetapkan.  
  - Hasil evaluasi divisualisasikan menggunakan **tfma.view.render_slicing_metrics** dan **tfma.addons.fairness.view.widget_view** untuk menampilkan metrik fairness dari model.  

In [24]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='real_news')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [25]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 15
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [26]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

In [27]:
tfma_result

EvalResult(slicing_metrics=[((), {'': {'': {'binary_accuracy': {'doubleValue': 0.9187817258883249}, 'loss': {'doubleValue': 0.24283470213413239}, 'example_count': {'doubleValue': 197.0}, 'auc': {'doubleValue': 0.9683806759278457}, 'false_positives': {'doubleValue': 10.0}, 'true_positives': {'doubleValue': 85.0}, 'false_negatives': {'doubleValue': 6.0}, 'true_negatives': {'doubleValue': 96.0}}}})], plots=[((), None)], attributions=[((), None)], config=model_specs {
  label_key: "real_news"
}
slicing_specs {
}
metrics_specs {
  metrics {
    class_name: "ExampleCount"
  }
  metrics {
    class_name: "AUC"
  }
  metrics {
    class_name: "FalsePositives"
  }
  metrics {
    class_name: "TruePositives"
  }
  metrics {
    class_name: "FalseNegatives"
  }
  metrics {
    class_name: "TrueNegatives"
  }
  metrics {
    class_name: "BinaryAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
      }
    }
  }
  model_names: ""
}
, data_location

# 10. Pusher: Model Deployment and Export

- **Pusher (Deployment Model)**:  
  - `Pusher` memastikan hanya model yang telah diberkati oleh evaluator yang akan dipindahkan ke direktori **SERVING_MODEL_DIR** untuk deployment, menggunakan **filesystem push destination**.  

- **Eksekusi Pipeline**:  
  - `interactive_context.run(...)` digunakan untuk menjalankan setiap komponen **model resolver, evaluator,** dan **pusher** dalam mode interaktif.  

In [28]:
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory=SERVING_MODEL_DIR))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 16
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## 10.1 Check End-Point Model Yang Digunakan Oleh TF-Serving

In [29]:
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/Fake_News_football_model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1741016862'}]}
